In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms.community import girvan_newman
from collections import defaultdict
import seaborn as sns
import random

In [2]:
from transformers import AutoModelForSequenceClassification
import torch

c:\Users\fedeu\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_json("C:\\Users\\fedeu\\Documents\\sim_project\\network_data.json")
df.head()

,comment_depth,interacts_with_depth,text_son,text_mother,son_date,mother_date,upvotes_son,upvotes_mother,author_son,author_mother
0,1,0,Same as it’s always been; entirely within reac...,Either for a two-state solution or a one-state...,2023-07-01,2023-07-01,34,15,KosherPigBalls,u/ANobodyBoy
1,2,1,beautifully said,Same as it’s always been; entirely within reac...,2023-07-01,2023-07-01,6,34,GrazingGeese,KosherPigBalls
2,2,1,Slightly more realistic than OP's username,Same as it’s always been; entirely within reac...,2023-07-02,2023-07-01,3,34,NumenSD,KosherPigBalls
3,2,1,Is this Schrödinger's peace cat ?,Same as it’s always been; entirely within reac...,2023-07-02,2023-07-01,1,34,Hsbsbhgdgdu,KosherPigBalls
4,1,0,We're more likely be heading to an escalation ...,Either for a two-state solution or a one-state...,2023-07-02,2023-07-01,5,15,Shachar2like,u/ANobodyBoy


In [4]:
total_corpus_son = df.groupby("author_son")["text_son"].apply(list).reset_index()

In [5]:
total_corpus_son["N_post"] = [len(total_corpus_son.loc[i,"text_son"]) for i in total_corpus_son.index]
total_corpus_son.sort_values("N_post",ascending = False, inplace = True)
total_corpus_son.reset_index(drop = True, inplace = True)

In [6]:
total_corpus_son

,author_son,text_son,N_post
0,AutoModerator,[> bullshit\n\n/u/JellyfishCosmonaut. Please a...,13058
1,JosephL_55,"[>And also, these guys appear darker than many...",4686
2,pinchasthegris,[You know the name palestine comes from the ol...,3095
3,JeffB1517,[Well in the case of a 1SS there is no Palesti...,2867
4,Queasy_Ad_7297,"[THIS ^, https://preview.redd.it/uh9xqay3ksxb1...",1853
...,...,...,...
33503,PurfectlySplendid,[I hope Karma gets to you eventually one day],1
33504,Purple-Eggplant-5429,[They have no choice. If Israel is invaded & c...,1
33505,Purple-Try8602,[Thank you.],1
33506,PurpleCactus69,"[Hamas gunned down kids at the festival, non o...",1


In [7]:
total_corpus_mother = df.groupby("author_mother")["text_mother"].apply(list).reset_index()
total_corpus_mother["N_post"] = [len(total_corpus_mother.loc[i,"text_mother"]) for i in total_corpus_mother.index]
total_corpus_mother.sort_values("N_post",ascending = False, inplace = True)
total_corpus_mother.reset_index(drop = True, inplace = True)

In [8]:
total_corpus = pd.merge(total_corpus_son,total_corpus_mother, how = "outer", left_on = "author_son", right_on = "author_mother")
total_corpus

,author_son,text_son,N_post_x,author_mother,text_mother,N_post_y
0,AutoModerator,[> bullshit\n\n/u/JellyfishCosmonaut. Please a...,13058.0,AutoModerator,[/u/botbot_16. 'Hitler' Casual comments and an...,779.0
1,JosephL_55,"[>And also, these guys appear darker than many...",4686.0,JosephL_55,"[>And also, these guys appear darker than many...",4126.0
2,pinchasthegris,[You know the name palestine comes from the ol...,3095.0,pinchasthegris,[You know the name palestine comes from the ol...,1979.0
3,JeffB1517,[Well in the case of a 1SS there is no Palesti...,2867.0,JeffB1517,[Well in the case of a 1SS there is no Palesti...,1887.0
4,Queasy_Ad_7297,"[THIS ^, https://preview.redd.it/uh9xqay3ksxb1...",1853.0,Queasy_Ad_7297,"[1) The PLO has pissed off Kuwait, Iraq, Jorda...",1031.0
...,...,...,...,...,...,...
35095,NaN,NaN,NaN,Reasonable_Set_2115,[[removed]],1.0
35096,NaN,NaN,NaN,Ok-Organization8454,[Looking for astonishing stories? I have some....,1.0
35097,NaN,NaN,NaN,Ok-Significance-3351,[[removed]],1.0
35098,NaN,NaN,NaN,Ok_Interaction_5701,[In the last years i somewhat supported palest...,1.0


In [9]:
total_corpus["text"] = total_corpus["text_son"] + total_corpus["text_mother"]
total_corpus["N_post"] = total_corpus["N_post_x"] + total_corpus["N_post_x"]

In [10]:
total_corpus.drop(["text_son","text_mother","N_post_x","N_post_y","author_mother"],axis = 1, inplace = True)

In [11]:
total_corpus = total_corpus.dropna().reset_index(drop = True).rename(columns = {"author_son" : "author"})

In [12]:
total_corpus

,author,text,N_post
0,AutoModerator,[> bullshit\n\n/u/JellyfishCosmonaut. Please a...,26116.0
1,JosephL_55,"[>And also, these guys appear darker than many...",9372.0
2,pinchasthegris,[You know the name palestine comes from the ol...,6190.0
3,JeffB1517,[Well in the case of a 1SS there is no Palesti...,5734.0
4,Queasy_Ad_7297,"[THIS ^, https://preview.redd.it/uh9xqay3ksxb1...",3706.0
...,...,...,...
25108,PulpFreedom,[Looks like it’s causing other Muslim nations ...,2.0
25109,Punchalibtard_,"[Gaza is a racist, apartheid society that has ...",2.0
25110,Pure_Language_5757,[The innocent citizens who are affected by thi...,2.0
25111,Pure_Seaweed9673,[Look into occupation. That’s the core issue. ...,2.0


In [13]:
total_corpus = total_corpus.drop(index=0).reset_index(drop = True)

In [14]:
total_corpus

,author,text,N_post
0,JosephL_55,"[>And also, these guys appear darker than many...",9372.0
1,pinchasthegris,[You know the name palestine comes from the ol...,6190.0
2,JeffB1517,[Well in the case of a 1SS there is no Palesti...,5734.0
3,Queasy_Ad_7297,"[THIS ^, https://preview.redd.it/uh9xqay3ksxb1...",3706.0
4,avbitran,[It sucks honestly I hoped people will wake up...,3332.0
...,...,...,...
25107,PulpFreedom,[Looks like it’s causing other Muslim nations ...,2.0
25108,Punchalibtard_,"[Gaza is a racist, apartheid society that has ...",2.0
25109,Pure_Language_5757,[The innocent citizens who are affected by thi...,2.0
25110,Pure_Seaweed9673,[Look into occupation. That’s the core issue. ...,2.0


In [15]:
test = total_corpus.iloc[:1000,:]

In [16]:
test

,author,text,N_post
0,JosephL_55,"[>And also, these guys appear darker than many...",9372.0
1,pinchasthegris,[You know the name palestine comes from the ol...,6190.0
2,JeffB1517,[Well in the case of a 1SS there is no Palesti...,5734.0
3,Queasy_Ad_7297,"[THIS ^, https://preview.redd.it/uh9xqay3ksxb1...",3706.0
4,avbitran,[It sucks honestly I hoped people will wake up...,3332.0
...,...,...,...
995,per-sieve-al,[This is by far the most logical argument I've...,200.0
996,Longjumping-Pen-9487,[That’s not new \nArabs wants to take control ...,200.0
997,verocity1989,[If you were actually educated on the issue yo...,200.0
998,Legitimate_Course686,[So it's ok for Israelis to invade our territo...,200.0


In [17]:
# sample 50 posts for each person
test["text"] = test["text"].apply(lambda x: random.sample(x,50))

C:\Users\fedeu\AppData\Local\Temp\ipykernel_13756\2159963981.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["text"] = test["text"].apply(lambda x: random.sample(x,50))


In [18]:
import re
# BASIC PREPROCESSING, REMOVE \n, \t, """ and emails
def preprocess(x):
    x = re.sub(r"\\n","",x)
    x = re.sub(r"\\t","",x)
    x = re.sub(r'"', "",x)
    #x = re.sub(r"https?:\/\/.*?[\s+]","",x) # remove URLs
    x = re.sub(r"([a-zA-Z0-9_\-\.]+)@([a-zA-Z0-9_\-\.]+)\.([a-zA-Z]{2,5})","",x) # Remove emails
    return x

In [19]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [20]:
# ENCODING IsraelPalestine
X_test = test.text.tolist()
X_test = [[preprocess(post) for post in batch] for batch in X_test]

In [22]:
#LOAD THE MODEL
roberta = AutoModelForSequenceClassification.from_pretrained("C:\\Users\\fedeu\\Documents\\sim_project\\roberta_pred")

In [23]:
# ENCODE THE INPUT
encodings = []
for user in X_test:
  encodings.append(tokenizer(user,truncation = True, padding = True, return_tensors = "pt"))

CLASSIFICATION - cambia num_users per provare su meno utenti

In [24]:
final = []
num_users = 300
count = 0
with torch.no_grad():
  for user in range(num_users):
    enc = encodings[user]
    outputs = roberta(**enc)
    probs = torch.nn.functional.softmax(outputs.logits, dim = 1).numpy()
    final.append(np.mean(probs, axis = 0))
    count += 1
    if count % 25 == 0:
      print(f"Current iteration: {count}/{num_users}")



Current iteration: 25/300
Current iteration: 50/300
Current iteration: 75/300
Current iteration: 100/300
Current iteration: 125/300
Current iteration: 150/300
Current iteration: 175/300
Current iteration: 200/300
Current iteration: 225/300
Current iteration: 250/300
Current iteration: 275/300
Current iteration: 300/300


In [73]:
# first_20 = pd.DataFrame({"Pro-Israel": [user[0] for user in final], "Pro-Palestine" : [user[1] for user in final] })
# first_20.to_csv("/content/drive/MyDrive/NLP_PROJECT/first_20.csv")

In [25]:
final

[array([0.7666855 , 0.23331447], dtype=float32),
 array([0.9154867 , 0.08451342], dtype=float32),
 array([0.8651766, 0.1348233], dtype=float32),
 array([0.84055215, 0.15944777], dtype=float32),
 array([0.84420145, 0.15579858], dtype=float32),
 array([0.6544051, 0.3455949], dtype=float32),
 array([0.85905457, 0.1409456 ], dtype=float32),
 array([0.80866736, 0.1913327 ], dtype=float32),
 array([0.8221771 , 0.17782289], dtype=float32),
 array([0.8910128 , 0.10898709], dtype=float32),
 array([0.8012725 , 0.19872753], dtype=float32),
 array([0.83101374, 0.1689863 ], dtype=float32),
 array([0.8801081 , 0.11989198], dtype=float32),
 array([0.9026812 , 0.09731892], dtype=float32),
 array([0.60497534, 0.39502457], dtype=float32),
 array([0.77010155, 0.22989833], dtype=float32),
 array([0.84982824, 0.15017183], dtype=float32),
 array([0.75625557, 0.24374425], dtype=float32),
 array([0.96212655, 0.03787336], dtype=float32),
 array([0.8086976 , 0.19130243], dtype=float32),
 array([0.7922159 , 0.20

In [26]:
out_df = pd.DataFrame({"Pro-Israel" : [x[0] for x in final], "Pro-Palestine" : [x[1] for x in final]})
out_df.to_csv("classified_users.csv")

In [44]:
out_df['Pro-Palestine'] > 0.25

0      False
1      False
2      False
3      False
4      False
       ...  
295    False
296     True
297    False
298     True
299     True
Name: Pro-Palestine, Length: 300, dtype: bool

In [46]:
test.loc[298, :]

author                                     Ok_Pangolin_9134
text      [Oh the irony: Jews who have lost their home i...
N_post                                                476.0
Name: 298, dtype: object

In [37]:
test

,author,text,N_post
0,JosephL_55,[Then it can’t be apartheid. Apartheid is a ra...,9372.0
1,pinchasthegris,[Lebanon callapesd only after palestinians imm...,6190.0
2,JeffB1517,[You sound like a Liberal Zionist. Join J-Str...,5734.0
3,Queasy_Ad_7297,[I can’t help but think we’d have been much be...,3706.0
4,avbitran,[And that's what it's all about. Very well put...,3332.0
...,...,...,...
995,per-sieve-al,"[Careful now, I recieved a ban from this very ...",200.0
996,Longjumping-Pen-9487,"[What I don’t understand is , why people even ...",200.0
997,verocity1989,[They resorted to religious radicalism because...,200.0
998,Legitimate_Course686,"[LOOL AT THE EMOJIS, what are you blind. Their...",200.0
